In [1]:
##### DNN module

from keras.utils import to_categorical
from keras import backend as K
from keras.models import Model
from keras.applications import ResNet50
from keras.optimizers import Adagrad, RMSprop, Adam, SGD
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
##### Common moldule

import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import time
from PIL import Image
from scipy.interpolate import RegularGridInterpolator
import pandas as pd

In [3]:
##### Path

train_db = 'database/combined_train_mx_240.npy'
label_db = 'database/combined_label_mx_240.npy'

flower_li = list(map(os.path.basename, glob.glob('database/image_data/train/*')))
flower_mapping = {flower_li[i]: i for i in range(5)}

print(flower_mapping)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [4]:
##### Load data

flower_data   = np.load(train_db).astype('uint8')
flower_target = np.load(label_db).astype('uint8')

x_train, x_test, y_train, y_test = train_test_split(flower_data, flower_target, test_size=0.2, random_state=9527)

In [5]:
##### Dimenssion check

(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

((2258, 240, 240, 3), (565, 240, 240, 3), (2258,), (565,))

In [6]:
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 5 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [7]:
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (2258, 240, 240, 3)
2258 train samples
565 test samples


In [8]:
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 240, 240, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 240, 240, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 238, 238, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 238, 238, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 119, 119, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 119, 119, 128)    

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [11]:
augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

In [12]:
history = model.fit_generator(augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
70/70 [==============================] - 25s 363ms/step - loss: 1.6466 - accuracy: 0.2273 - val_loss: 1.6035 - val_accuracy: 0.2496
Epoch 2/10
70/70 [==============================] - 22s 320ms/step - loss: 1.6165 - accuracy: 0.2403 - val_loss: 1.6058 - val_accuracy: 0.2513
Epoch 3/10
70/70 [==============================] - 23s 322ms/step - loss: 1.5715 - accuracy: 0.3041 - val_loss: 1.7205 - val_accuracy: 0.2566
Epoch 4/10
70/70 [==============================] - 23s 323ms/step - loss: 1.4335 - accuracy: 0.3850 - val_loss: 1.3201 - val_accuracy: 0.4159
Epoch 5/10
70/70 [==============================] - 23s 325ms/step - loss: 1.3147 - accuracy: 0.4331 - val_loss: 1.3421 - val_accuracy: 0.4071
Epoch 6/10
70/70 [==============================] - 21s 301ms/step - loss: 1.3238 - accuracy: 0.4236 - val_loss: 1.3962 - val_accuracy: 0.3965
Epoch 7/10
70/70 [==============================] - 21s 293ms/step - loss: 1.2675 - accuracy: 0.4497 - val_loss: 1.2751 - val_accuracy: 0.4425

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.2251407787863131
Test accuracy: 0.484955757856369


In [ ]:
model.save('model/STD_datagen_1230_06.h5')

In [ ]:
##### Test on unknown sample

In [ ]:
def resampleRGI3d(input_mx, resize_to, dtype='float32'):
    # Ref: 10 3-A-7
    # Ver. 2.1
    # input_mx : numpy array, the original target matrix
    # resize_to: list or tuple with 3 int inside
    a, b, c = np.shape(input_mx)
    p, q, r = resize_to
    z_grid = np.linspace(0, p - 1, a)
    y_grid = np.linspace(0, q - 1, b)
    x_grid = np.linspace(0, r - 1, c)
    RGI = RegularGridInterpolator((z_grid, y_grid, x_grid), input_mx)
    z_grid_t2 = np.arange(p)
    y_grid_t2 = np.arange(q)
    x_grid_t2 = np.arange(r)
    meshgrid_para = np.meshgrid(z_grid_t2, y_grid_t2, x_grid_t2)
    RGI_mesh_mx = RGI((meshgrid_para[0], meshgrid_para[1], meshgrid_para[2]))
    RGI_mx = np.transpose(RGI_mesh_mx, axes=[1, 0, 2]).astype(dtype)
    return RGI_mx

def happy_time(start,stop):
    process_time = round(stop - start)
    ss = process_time % 60
    mm = process_time // 60 % 60
    hh = process_time // 3600
    duration = "Process time == {}s == {}H {}m {}s".format(process_time,hh,mm,ss)
    return duration

In [ ]:
path_unknown = glob.glob('database/image_data/test/*')
path_unknown[:5]

In [ ]:
#path_train_0 = glob.glob(os.path.join(path_train, list(flower_mapping.keys())[i], '*'))
#pic_store_dict = None
#pic_store_dict = {}

id_li = []
flower_class = []
loop_start = time.time()

for j in range(len(path_unknown)):
    serial = os.path.basename(path_unknown[j])[:-4]
    temp_pic = np.asarray(Image.open(path_unknown[j]))
    temp_shape = np.shape(temp_pic)
    if temp_shape[0] > 640:
        temp_pic = temp_pic[:640]
        temp_shape = np.shape(temp_pic)
    if temp_shape[1] > 640:
        temp_pic = temp_pic[:, :640]
        temp_shape = np.shape(temp_pic)
    if temp_shape[0] > 240:
        temp_pic = resampleRGI3d(temp_pic, (240, int(temp_shape[1]*240/temp_shape[0]), 3))
        temp_shape = np.shape(temp_pic)
    if temp_shape[1] > 240:
        temp_pic = resampleRGI3d(temp_pic, (int(temp_shape[0]*240/temp_shape[1]), 240, 3))
        temp_shape = np.shape(temp_pic)
    temp_pic = np.pad(temp_pic, ((0, 240 - temp_shape[0]), (0, 240 - temp_shape[1]), (0, 0)), 'constant', constant_values=0)
    temp_pic = np.expand_dims(temp_pic, axis = 0)
    pred = model.predict(temp_pic)[0]
    id_li.append(serial)
    flower_class.append(list(pred).index(max(pred)))
    if (j+1) % 200 == 0:
        print(j+1, 'files done.', happy_time(loop_start, time.time()))
    #pic_store_dict[serial] = temp_pic
#stack_a = np.stack([pic_store_dict[x] for x in list(pic_store_dict.keys())], axis=0)
#total_dict[sub_folder_li[i]] = stack_a

In [ ]:
pred_result_df = pd.DataFrame(columns=['id', 'flower_class'])
pred_result_df['id'] = id_li
pred_result_df['flower_class'] = flower_class
pred_result_df

In [ ]:
pred_result_df.to_csv('model/pred_result_STD_datagen_1230_06.csv', index=False)

In [ ]:
#####
#####
#####